In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

DIVERSITY_UTILITY = float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = float(os.environ.get('FRAC_ADMIT'))

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_minority','T_majority','Y']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_minority,T_majority,Y
999835,0,11,0.00,11,11,0
999838,0,11,0.00,7,11,0
999837,0,11,0.00,8,11,0
999836,0,11,0.00,11,11,0
999886,0,12,0.02,11,12,0
...,...,...,...,...,...,...
999913,1,98,1.25,98,115,1
999914,1,98,1.25,98,115,1
999925,1,109,1.25,109,126,1
998845,1,97,1.25,97,112,1


In [4]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [5]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [6]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [7]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [8]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
2,221,2,0,11,0.00,4
1,219,1,0,10,0.08,1
3,220,3,0,12,0.08,4
0,218,0,0,9,0.10,1
120,217,120,1,8,0.32,1
...,...,...,...,...,...,...
51,4,51,0,60,10970.40,18284
47,3,47,0,56,10985.52,21126
48,2,48,0,57,11030.04,20426
50,1,50,0,59,11169.64,19258


### Setup optimization problem 

In [9]:
from ortools.linear_solver import pywraplp


In [10]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [11]:
dff

,level_0,index,R,T,ml_outcomes,N
0,218,0,0,9,0.10,1
1,219,1,0,10,0.08,1
2,221,2,0,11,0.00,4
3,220,3,0,12,0.08,4
4,211,4,0,13,1.00,10
...,...,...,...,...,...,...
217,203,217,1,107,1.25,1
218,204,218,1,109,1.25,1
219,205,219,1,111,1.25,1
220,212,220,1,113,0.92,1


In [12]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [13]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [14]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total admits cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

249999


In [15]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [16]:
from collections import Counter

def convertListToProb(raw_list):
    counts = dict(Counter(raw_list))
    probs = {}
    for test_score in counts:
        probs[test_score]  = counts[test_score]/float(len(raw_list))
    return [(probs[t], t) for t in probs]

In [17]:
T_minoritys_list = df[df['R']==0][['T','T_minority']].groupby('T')['T_minority'].apply(list).reset_index(name='T_minoritys')
T_minoritys_list['probs'] = T_minoritys_list['T_minoritys'].apply(convertListToProb)
didntexist = 0
exists = 0
for ix, row in T_minoritys_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    majority_T = row['T']
    minoritys_Ts = row['probs']        
    cf_fair_stratum.SetCoefficient(vars_cache[(0.0, majority_T)], -1.0)
    for prob in minoritys_Ts:
        if (1.0, prob[1]) not in vars_cache:
            vars_cache[(1.0, prob[1])] = solver.NumVar(0.0, 1.0, str((1.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
        cf_fair_stratum.SetCoefficient(vars_cache[(1.0, prob[1])], prob[0])
didntexist,exists

(3, 978)

In [18]:
T_majoritys_list = df[df['R']==1][['T','T_majority']].groupby('T')['T_majority'].apply(list).reset_index(name='T_majoritys')
T_majoritys_list['probs'] = T_majoritys_list['T_majoritys'].apply(convertListToProb)
didntexist = 0
exists = 0

for ix, row in T_majoritys_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    minority_T = row['T']
    majority_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(1.0, minority_T)], -1.0)
    for prob in majority_Ts:
        if (0.0, prob[1]) not in vars_cache:
            vars_cache[(0.0, prob[1])] = solver.NumVar(0.0, 1.0, str((0.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
            
        cf_fair_stratum.SetCoefficient(vars_cache[(0.0, prob[1])], prob[0])
didntexist,exists  

(3, 885)

In [19]:
solver.NumConstraints()

223

## Solve linear program

In [20]:
solver.ABNORMAL

4

In [21]:
status = solver.Solve()


In [22]:
status

0

In [23]:
solver.OPTIMAL

0

In [24]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [25]:
df_decisions

,row_id,decision
0,0,0.249986
1,1,0.249986
2,2,0.249986
3,3,0.249986
4,4,0.249986
...,...,...
217,217,1.000000
218,218,1.000000
219,219,1.000000
220,220,1.000000


In [26]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
49,0,49,0,58,11233.60,20060,49,0.249986
50,1,50,0,59,11169.64,19258,50,0.249986
48,2,48,0,57,11030.04,20426,48,0.249986
47,3,47,0,56,10985.52,21126,47,0.249986
51,4,51,0,60,10970.40,18284,51,0.249986
...,...,...,...,...,...,...,...,...
120,217,120,1,8,0.32,1,120,0.249986
0,218,0,0,9,0.10,1,0,0.249986
1,219,1,0,10,0.08,1,1,0.249986
3,220,3,0,12,0.08,4,3,0.249986


In [27]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,ml_outcomes,N,row_id,decision
119,215,119,1,7,0.50,2,119,0.249986
120,217,120,1,8,0.32,1,120,0.249986
0,218,0,0,9,0.10,1,0,0.249986
121,207,121,1,9,1.00,4,121,0.249986
1,219,1,0,10,0.08,1,1,0.249986
...,...,...,...,...,...,...,...,...
114,200,114,0,123,2.00,2,114,0.749986
115,209,115,0,124,1.00,1,115,1.000000
116,199,116,0,125,2.00,2,116,1.000000
117,208,117,0,127,1.00,1,117,1.000000


In [28]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
49,0,49,0,58,11233.60,20060,49,0.249986
50,1,50,0,59,11169.64,19258,50,0.249986
48,2,48,0,57,11030.04,20426,48,0.249986
47,3,47,0,56,10985.52,21126,47,0.249986
51,4,51,0,60,10970.40,18284,51,0.249986
...,...,...,...,...,...,...,...,...
120,217,120,1,8,0.32,1,120,0.249986
0,218,0,0,9,0.10,1,0,0.249986
1,219,1,0,10,0.08,1,1,0.249986
3,220,3,0,12,0.08,4,3,0.249986


In [29]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [30]:
len(xxx)

222

In [31]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT


FRAC_minority_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()


In [32]:
FRAC_minority_POLICY, SUM_BP_POLICY

(0.3301010583060487, 110675.59900282591)

In [33]:
FRAC_RANDOM_POLICY, SUM_BP_RAND_POLICY

(0.3312554264977134, 110687)

In [34]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Fairness',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()


In [35]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.24999924999925005